# 🚀 PromptOps with LangSmith and Azure OpenAI

This Jupyter Notebook provides a step-by-step guide on implementing **Prompt Operations (PromptOps)** using **LangSmith** for prompt management, **LangChain** for orchestration, and **Azure OpenAI** for content generation. We will cover the entire lifecycle: prompt registration, modification, loading, generation and deletion.

## Prerequisites

Before running the code, ensure you have the following environment variables set up. These are crucial for connecting to LangSmith and Azure OpenAI.

| Variable | Purpose | Example Value |
| :--- | :--- | :--- |
| `LANGSMITH_API_KEY` | Authentication for LangSmith | `lsv2_sk_...` |
| `LANGSMITH_TRACING` | Name of the LangSmith project for tracing | `true` |
| `AZURE_OPENAI_ENDPOINT` | Your Azure OpenAI Service endpoint | `https://your-resource.openai.azure.com/` |
| `AZURE_OPENAI_API_KEY` | Your Azure OpenAI API key | `abcdef123456...` |
| `OPENAI_API_VERSION` | Azure OpenAI API version | `2024-02-01` |

The Azure OpenAI deployment name used in this tutorial is **`gpt-4.1-mini`**, as specified in the requirements.

In [1]:
!pip install langchain langchain-openai langchain-core langsmith evaluate openai python-dotenv rouge_score --quiet

In [ ]:
# 1. Setup and Imports
import os
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from evaluate import load
import json

In [ ]:
# --- Environment Variable Setup (Replace with your actual values or set them in your environment) ---
# NOTE: In a real environment, these should be set externally for security.
os.environ["LANGSMITH_API_KEY"] = os.environ.get("LANGSMITH_API_KEY", "xxxxxxxxxxxxxxxxx")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get("AZURE_OPENAI_ENDPOINT", "https://eastus.api.cognitive.microsoft.com/")
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get("AZURE_OPENAI_API_KEY", "xxxxxxxxxxxxx")
os.environ["OPENAI_API_VERSION"] = os.environ.get("OPENAI_API_VERSION", "2024-08-01-preview")
os.environ["LANGSMITH_TRACING"] = "true"


In [ ]:
# Initialize LangSmith Client
client = Client()
print("LangSmith Client Initialized.")

In [9]:
# Initialize Azure OpenAI Model
try:
    llm = AzureChatOpenAI(
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        azure_deployment="gpt-4.1-mini",
        temperature=0.7
    )
    print("AzureChatOpenAI Model Initialized with deployment: gpt-4.1-mini")
except Exception as e:
    print(f"Warning: Could not initialize AzureChatOpenAI. Please ensure all environment variables are set correctly. Error: {e}")
    llm = None # Set to None if initialization fails


LangSmith Client Initialized.
AzureChatOpenAI Model Initialized with deployment: gpt-4.1-mini


## 1. Prompt Registration (Pushing a Prompt)

LangSmith allows you to register and version your prompts programmatically using the `client.push_prompt()` method. We will create a simple prompt for generating short, professional summaries of technical concepts.

The prompt name we will use is **`technical-summary-generator`**.

In [10]:
# Define the initial prompt (Version 1)
prompt_name = "technical-summary-generator"
initial_template = (
    "You are a professional technical writer. Your task is to write a concise, one-paragraph summary of the following technical concept: {concept}. "
    "The summary must be easy to understand for a non-technical audience."
)

initial_prompt = ChatPromptTemplate.from_template(initial_template)

# Push the initial prompt to LangSmith
try:
    url = client.push_prompt(prompt_name, object=initial_prompt)
    print(f"Successfully registered prompt '{prompt_name}' (Version 1).")
    print(f"View in LangSmith: {url}")
except Exception as e:
    print(f"Error pushing prompt: {e}")


Successfully registered prompt 'technical-summary-generator' (Version 1).
View in LangSmith: https://smith.langchain.com/prompts/technical-summary-generator/3528168c?organizationId=d902fd7a-b325-505a-adcb-ea98ad22246a


## 2. Prompt Modification (Pushing a New Version)

To modify the prompt, we simply define a new version and use `client.push_prompt()` again with the same name. LangSmith automatically handles the versioning, creating a new commit for the change.

**Modification**: We will add a constraint to include a real-world example in the summary.

In [11]:
# Define the modified prompt (Version 2)
modified_template = (
    "You are a professional technical writer. Your task is to write a concise, one-paragraph summary of the following technical concept: {concept}. "
    "The summary must be easy to understand for a non-technical audience and **must include a simple, real-world example**."
)

modified_prompt = ChatPromptTemplate.from_template(modified_template)

# Push the modified prompt to LangSmith
try:
    url = client.push_prompt(prompt_name, object=modified_prompt)
    print(f"Successfully updated prompt '{prompt_name}' (Version 2).")
    print(f"View in LangSmith: {url}")
except Exception as e:
    print(f"Error pushing modified prompt: {e}")


Successfully updated prompt 'technical-summary-generator' (Version 2).
View in LangSmith: https://smith.langchain.com/prompts/technical-summary-generator/0438a4a7?organizationId=d902fd7a-b325-505a-adcb-ea98ad22246a


## 3. Loading the Prompt (Pulling a Prompt)

We can load the latest version of the prompt using `client.pull_prompt()`. This ensures our application always uses the most up-to-date, tested prompt from the hub.

In [12]:
# Pull the latest prompt from LangSmith
try:
    latest_prompt = client.pull_prompt(prompt_name)
    print(f"Successfully pulled latest prompt: '{prompt_name}'")
    print("\n--- Pulled Prompt Template ---")
    print(latest_prompt.messages[0].prompt.template)
except Exception as e:
    print(f"Error pulling prompt: {e}")
    latest_prompt = None


Successfully pulled latest prompt: 'technical-summary-generator'

--- Pulled Prompt Template ---
You are a professional technical writer. Your task is to write a concise, one-paragraph summary of the following technical concept: {concept}. The summary must be easy to understand for a non-technical audience and **must include a simple, real-world example**.


## 4. Content Generation with Azure OpenAI

Now we combine the loaded prompt with our `AzureChatOpenAI` model to create a simple LangChain Expression Language (LCEL) chain for content generation. The execution of this chain will automatically be traced and logged in LangSmith, linking the generated output back to the specific prompt version used.

In [13]:
if llm and latest_prompt:
    # Create the LCEL chain: Prompt -> LLM -> Output Parser
    chain = latest_prompt | llm | StrOutputParser()

    # Define the input concept
    input_concept = "Quantum Computing"

    print(f"Generating summary for: {input_concept}...")

    # Invoke the chain
    try:
        response = chain.invoke({"concept": input_concept})
        print("\n--- Generated Summary ---")
        print(response)
        print("\nCheck LangSmith for the trace of this run!")
    except Exception as e:
        print(f"Error during content generation: {e}")
        response = "Generation Failed"
else:
    response = "Generation Skipped due to setup errors."
    print(response)


Generating summary for: Quantum Computing...

--- Generated Summary ---
Quantum computing is a new type of computing that uses the strange behavior of tiny particles, called quantum bits or qubits, to solve problems much faster than regular computers. Unlike traditional bits that are either 0 or 1, qubits can be both at the same time, allowing quantum computers to explore many possibilities simultaneously. Imagine trying to find the fastest route through a city with many roads: a regular computer checks each path one by one, but a quantum computer can consider all routes at once, quickly finding the best option. This technology has the potential to revolutionize fields like medicine, finance, and cryptography by solving complex problems that are currently impossible for ordinary computers.

Check LangSmith for the trace of this run!


## 5. List, delete, and like prompts

You can also list, delete, and like/unlike prompts using the list prompts, delete prompt, like prompt and unlike prompt methods.

In [ ]:
# List all prompts in my workspace
prompts = client.list_prompts()

# List my private prompts that include "joke"
prompts = client.list_prompts(query="summarize", is_public=False)

# Delete a prompt
client.delete_prompt(prompt_name)


In [18]:
# Like a prompt
client.like_prompt("efriis/my-first-prompt")

{'likes': 2}

In [19]:
# Unlike a prompt
client.unlike_prompt("efriis/my-first-prompt")

{'likes': 1}

## Conclusion

This tutorial demonstrated a complete PromptOps workflow:

1.  **Registration & Modification**: Using `client.push_prompt()` to version control prompts in LangSmith.
2.  **Loading**: Using `client.pull_prompt()` to ensure the application uses the latest, approved version.
3.  **Generation**: Integrating the prompt with `AzureChatOpenAI` via a LangChain LCEL chain, with automatic tracing to LangSmith.

This workflow is the foundation for robust, and iterative development of LLM applications.